In [1]:
import pandas as pd

# only columns name, color, and sku
art = pd.read_csv("../data/processed/articles_clean.csv", usecols=["sku", "name", "color", "name.1", "size", "status", "audience", "category"], dtype=str)

art.head()

,sku,status,name,name.1,color,size,audience,category
0,000DIV,active,NaN,unknown,unknown,unknown,unknown,unknown
1,052743,inactive,Lakan örngott,unknown,blå,unknown,unknown,unknown
2,055522,active,Beskrivning Tröja,Gjestal Garn,unknown,unknown,dam,Tröjor
3,055573,active,Beskrivning Luva,Novita,unknown,unknown,dam,"Mössor & hattar,Mönster"
4,055575,active,Beskrivning Vantar,Novita,unknown,unknown,dam,Vantar
